## Fixed Data:

In [21]:
pip install fredapi

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy>=1.15.4 in c:\zacharyyang\lib\site-packages (from pandas->fredapi) (1.19.2)



In [1]:
import json
import numpy as np
import requests
import pandas as pd
from fredapi import Fred

API_KEY = '1vGb0FOsg2hEIGrEhpueLCcWd1Y'
FRED_API_KEY = 'b13a905b98b5dfc1f3910d6a0d875ae3'

rootUrl = 'https://api.glassnode.com'

symbolList = ['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER', 'BAL', 'BAND', 'BAT', 'BIX', 'BNT', 'BOND', 'BRD', 'BUSD', 'BZRX', 'CELR', 'CHSB', 'CND', 'COMP', 'CREAM', 'CRO', 'CRV', 'CVC', 'CVP', 'DAI', 'DDX', 'DENT', 'DGX', 'DHT', 'DMG', 'DODO', 'DOUGH', 'DRGN', 'ELF', 'ENG', 'ENJ', 'EURS', 'FET', 'FTT', 'FUN', 'GNO', 'GUSD', 'HEGIC', 'HOT', 'HPT', 'HT', 'HUSD', 'INDEX', 'KCS', 'LAMB', 'LBA', 'LDO', 'LEO', 'LINK', 'LOOM', 'LRC', 'MANA', 'MATIC',
              'MCB', 'MCO', 'MFT', 'MIR', 'MKR', 'MLN', 'MTA', 'MTL', 'MX', 'NDX', 'NEXO', 'NFTX', 'NMR', 'Nsure', 'OCEAN', 'OKB', 'OMG', 'PAY', 'PERP', 'PICKLE', 'PNK', 'PNT', 'POLY', 'POWR', 'PPT', 'QASH', 'QKC', 'QNT', 'RDN', 'REN', 'REP', 'RLC', 'ROOK', 'RPL', 'RSR', 'SAI', 'SAN', 'SNT', 'SNX', 'STAKE', 'STORJ', 'sUSD', 'SUSHI', 'TEL', 'TOP', 'UBT', 'UMA', 'UNI', 'USDC', 'USDK', 'USDP', 'USDT', 'UTK', 'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI', 'ZRX']

symbolListShort = ['BTC', 'ETH', 'LTC']

endPoints = {
    'Price': '/v1/metrics/market/price_usd_close',
    'MarketCap': '/v1/metrics/market/marketcap_usd',
    'TransactionCount': '/v1/metrics/transactions/count',
    'ActiveAddress': '/v1/metrics/addresses/active_count',
}

In [48]:
fred = Fred(api_key=FRED_API_KEY)
Risk_free_rate = fred.get_series('DGS1MO', frequency = 'w')
Risk_free_rate = Risk_free_rate.to_frame()
Risk_free_rate = Risk_free_rate.reset_index()
Risk_free_rate = Risk_free_rate.rename(columns={'index':'Date', 0:'RiskFree'})

## Get Multiple Attributes for Multiple Coins:

In [3]:
### Define a function to pull data according to input values(coin, feature)
### After running this method, it should return pulled data, and also store it in memory named as "coin_feature"

def PullData(coin, feature):
    ##print(f"Pulling {coin}'s {feature} data")

    # Assemble full request url and get responses:
    reqUrl = rootUrl + endPoints[feature]
    res = requests.get(reqUrl,
                       params={'a': coin, 'api_key': API_KEY, 'i':'1w'})
    data_raw = pd.read_json(res.text, convert_dates=['t'])

    # Rename column "v" according to features
    data = data_raw.rename(columns={"v" : f"{feature}", 't':'Date'})

    # Turn renamed dataframe into a global variable for future using
    name = f"{coin}_{feature}"
    globals()[name] = data

    # Return a dataframe
    return data

In [32]:
### Get all features(listed in endPoints) of all coins(listed in symbolListShort) 

from functools import reduce

for i in symbolList:
    ##print(f"Start Pulling {i}'s data:")

    # Preparation for merging
    frameName = f"{i}_Frames"
    locals()[frameName] = []

    # Get data
    for j in list(endPoints.keys()):
        locals()[frameName].append(PullData(i, j))

    # Merge to single dataframe
    ##print('Merging Dataframes...')
    dfName = f"{i}_Raw"
    locals()[dfName] = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='inner'), locals()[frameName])
    locals()[dfName] = locals()[dfName].merge(Risk_free_rate)

##    print(f"{i} data successfully pulled\n")
##print('\nALL DATA SUCCESSFULLY PULLED!')

Pulling BTC's Price data
Pulling BTC's MarketCap data
Pulling BTC's TransactionCount data
Pulling BTC's ActiveAddress data
Pulling ETH's Price data
Pulling ETH's MarketCap data
Pulling ETH's TransactionCount data
Pulling ETH's ActiveAddress data
Pulling LTC's Price data
Pulling LTC's MarketCap data
Pulling LTC's TransactionCount data
Pulling LTC's ActiveAddress data
Pulling AAVE's Price data
Pulling AAVE's MarketCap data
Pulling AAVE's TransactionCount data
Pulling AAVE's ActiveAddress data
Pulling ABT's Price data
Pulling ABT's MarketCap data
Pulling ABT's TransactionCount data
Pulling ABT's ActiveAddress data
Pulling AMPL's Price data
Pulling AMPL's MarketCap data
Pulling AMPL's TransactionCount data
Pulling AMPL's ActiveAddress data
Pulling ANT's Price data
Pulling ANT's MarketCap data
Pulling ANT's TransactionCount data
Pulling ANT's ActiveAddress data
Pulling ARMOR's Price data
Pulling ARMOR's MarketCap data
Pulling ARMOR's TransactionCount data
Pulling ARMOR's ActiveAddress data


In [37]:
AMPL_Raw.head()

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree
0,2019-06-24,1.935700,0.0,0,572,2.12
1,2019-07-01,1.255103,0.0,0,158,2.17
2,2019-07-08,1.556803,0.0,0,145,2.23
3,2019-07-15,1.313191,0.0,0,1152,2.17
4,2019-07-22,0.998086,0.0,0,101,2.13


In [38]:
NET = pd.DataFrame({'Date':[]})
NET.Date = BTC_Raw.Date

for i in symbolList:
    copy1 = f"{i}1"
    locals()[copy1] = locals()[dfName].copy()
    copy2 = f"{i}2"
    locals()[copy2] = locals()[dfName].copy()
    locals()[copy2] = locals()[copy2].shift()

    df_NET = f"{i}_NET"
    locals()[df_NET] = pd.DataFrame({'Date':[],f"{i}":[]})
    locals()[df_NET][f"{i}"] = np.log(locals()[copy1]['ActiveAddress'])-np.log(locals()[copy2]['ActiveAddress'])
    locals()[df_NET]['Date'] = locals()[copy1]['Date']

    NET = NET.merge(locals()[df_NET], left_on='Date', right_on='Date', how='outer')


In [40]:
NET.head()

,Date,BTC,ETH,LTC,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,...,VERI,WaBi,WAX,WBTC,WETH,wNXM,WTC,YAM,YFI,ZRX
0,2010-07-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-07-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
C5 = pd.DataFrame({'Date':[]})
C5.Date = BTC_Raw.Date

C5['mean'] = NET.mean(axis=0, numeric_only= True)
C5['30Percentile'] = NET.quantile(q=0.3, numeric_only= True, axis = 0)
C5['70Percentile'] = NET.quantile(q=0.7, numeric_only= True, axis = 0)


In [46]:
C5.head()

,Date,mean,30Percentile,70Percentile
0,2010-07-12,NaN,NaN,NaN
1,2010-07-19,NaN,NaN,NaN
2,2010-07-26,NaN,NaN,NaN
3,2010-08-02,NaN,NaN,NaN
4,2010-08-09,NaN,NaN,NaN


In [34]:
LTC_All.head()

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,NET
0,2013-05-13,3.149100,5.601635e+07,45636,63963,0.02,NaN
1,2013-05-20,3.150000,5.668063e+07,40096,57330,0.01,-0.109481
2,2013-05-27,2.830000,5.150006e+07,34165,49767,NaN,-0.141472
3,2013-06-03,2.348333,4.322178e+07,32639,48009,0.03,-0.035964
4,2013-06-10,2.150000,3.999957e+07,31174,49237,0.04,0.025257
